In [4]:
#-*- coding: utf-8 -*-
import urllib
from bs4 import BeautifulSoup                    #beautifulsoup
from selenium import webdriver                   #selenium
from selenium.webdriver.support.ui import Select #select
import time

#1.타자의 경기 종류별 기록테이블이 다름
#경기 타석  타수  득점 안타   2루타 3루타 홈런  루타 타점   희생번 희생플 도루  도루실 볼넷  사구  삼진  병살타 실책
#G    PA    AB    R    H      2B    3B    HR    TB   RBI    SAC    SF                                                 #option==0
#G    PA    AB         H      2B    3B    HR         RBI                  SB    CS     BB    HBP   SO    GDP    E     #그외
#2.투수페이지가 로딩은 되지만 아직 테스트해보지 않음.
#투수일 경우 포지션select를 실행하지 않도록 해놓긴 했지만 확실치 않음
#게다가 현재 코드라면 투수일 경우 포지션select부분 안의 데이터 추출부분도 동작하지 않음

File = open("KBOTest.txt", 'a') #이어쓰기방식으로 텍스트파일 열기

SeasonList = ['2012', '2013', '2014', '2015', '2016']                     #연도별
SeriesList = ['0', '1', '4', '3', '5', '7']                               #경기 종류별
TeamList = [['WO', 'OB', 'LT', 'SS', 'HH', 'HT', 'LG', 'SK'],             #팀별(~2012까지의 팀)
            ['WO', 'OB', 'LT', 'SS', 'HH', 'HT', 'LG', 'NC', 'SK'],       #    (2013~2014까지의 팀)
            ['WO', 'OB', 'LT', 'SS', 'HH', 'HT', 'KT', 'LG', 'NC', 'SK']] #    (2014부터의 팀)
TeamNum = 0
PosList = ['2', '3,4,5,6', '7,8,9']                                       #포지션별(타자만 해당)

driver = webdriver.Chrome("C:\Users\Kim Jin Hak\Desktop\chromedriver.exe")        #크롬드라이버로딩?
SiteURL = ['http://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx',  #기록실의 타자부분 첫페이지 url
           'http://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx'] #기록실의 투수부분 첫페이지 url
for url in SiteURL:
    driver.get(url)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    #모든 경우에서 select의 위치는 id로 찾음
    #각 경우마다 차례대로 select->option의 value값을 변경하고, 바뀐 웹페이지에서 소스를 다시 가져와 데이터를 뽑아냄
    #지연시간을 주지 않으면 웹페이지가 로딩되기 전에 다음 코드로 넘어가 잘못된 데이터를 가져옴
    for season in SeasonList: #연도별 select->option의 value값 변경
        selectSeason = Select(driver.find_element_by_id("cphContainer_cphContents_ddlSeason_ddlSeason"))
        selectSeason.select_by_value(season)
        time.sleep(5)
        for series in SeriesList: #경기 종류별 select->option의 value값 변경
            selectSeries = Select(driver.find_element_by_id("cphContainer_cphContents_ddlSeries_ddlSeries"))
            selectSeries.select_by_value(series)
            time.sleep(5)
            #연도별로 팀의 수가 다름
            if(season < '2013'):
                TeamNum = 0
            elif(season <'2015'):
                 TeamNum = 1
            else:
                TeamNum = 2
            for team in TeamList[TeamNum]:  #팀별 select->option의 value값 변경
                selectTeam = Select(driver.find_element_by_id("cphContainer_cphContents_ddlTeam_ddlTeam"))
                selectTeam.select_by_value(team)
                time.sleep(5)
                #타자냐 투수냐에 따라 포지션이 있고 없음
                if(url == ('http://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx' or 'http://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx')):
                    for pos in PosList: #포지션별 select->option의 value값 변경(타자만)
                        selectPos = Select(driver.find_element_by_id("cphContainer_cphContents_ddlPos_ddlPos"))
                        selectPos.select_by_value(pos)
                        time.sleep(5)              

                        #달라진 웹페이지 재추출
                        html_r = driver.page_source           
                        soup_r = BeautifulSoup(html_r, "lxml") 

                        #현재 뽑아오는 데이터의 종류?를 보여줌 
                        SeasonName = soup_r.find('option', attrs={'value':season})
                        SeriesName = soup_r.find('option', attrs={'value':series})
                        TeamName = soup_r.find('option', attrs={'value':team})
                        PosName = soup_r.find('option', attrs={'value':pos})
                        print SeasonName.find(text=True) + SeriesName.find(text=True) + TeamName.find(text=True) + PosName.find(text=True)
                        File.write(SeasonName.find(text=True).encode('utf-8') + SeriesName.find(text=True).encode('utf-8') + TeamName.find(text=True).encode('utf-8') + PosName.find(text=True).encode('utf-8'))
                        File.write('\n')

                        #table태그에 들어있는 기록 데이터를 뽑아옴
                        table      = soup_r.find('table', attrs={'class':'tData01 tt'})
                        table_body = table.find('tbody')                             
                        rows       = table_body.find_all('tr')                        
                        for row in rows:
                            cells    = row.findAll('td')       
                            Rank     = cells[0].find(text=True)
                            Name     = cells[1].find(text=True)
                            teamName = cells[2].find(text=True)
                            AVG      = cells[3].find(text=True)
                            G        = cells[4].find(text=True)
                            PA       = cells[5].find(text=True) 
                            AB       = cells[6].find(text=True) 
                            R        = cells[7].find(text=True)
                            H        = cells[8].find(text=True)
                            _2B      = cells[9].find(text=True) 
                            _3B      = cells[10].find(text=True) 
                            HR       = cells[11].find(text=True) 
                            TB       = cells[12].find(text=True) 
                            RBI      = cells[13].find(text=True) 
                            SAC      = cells[14].find(text=True) 
                            SF       = cells[15].find(text=True) 
                            File.write("{0:4s}| {1:11s}| {2:4s}| {3:4s}| {4:4s}| {5:4s}| {6:4s}| {7:4s}| {8:4s}| {9:4s}| {10:4s}| {11:4s}| {12:4s}| {13:4s}| {14:4s}| {15:4s}\n"
                                    .format(Rank, Name.encode('utf-8'), teamName.encode('utf-8'), AVG, G, PA, AB, R, H, _2B, _3B, HR, TB, RBI, SAC, SF))
                        File.write('\n')
File.close()
print 'finish'

2012KBO 정규시즌넥센포수
2012KBO 정규시즌넥센내야수
2012KBO 정규시즌넥센외야수
2012KBO 정규시즌두산포수
2012KBO 정규시즌두산내야수
2012KBO 정규시즌두산외야수
2012KBO 정규시즌롯데포수
2012KBO 정규시즌롯데내야수
2012KBO 정규시즌롯데외야수
2012KBO 정규시즌삼성포수
2012KBO 정규시즌삼성내야수
2012KBO 정규시즌삼성외야수
2012KBO 정규시즌한화포수
2012KBO 정규시즌한화내야수
2012KBO 정규시즌한화외야수
2012KBO 정규시즌KIA포수
2012KBO 정규시즌KIA내야수
2012KBO 정규시즌KIA외야수
2012KBO 정규시즌LG포수
2012KBO 정규시즌LG내야수
2012KBO 정규시즌LG외야수
2012KBO 정규시즌SK포수
2012KBO 정규시즌SK내야수
2012KBO 정규시즌SK외야수
2012KBO 시범경기넥센포수
2012KBO 시범경기넥센내야수
2012KBO 시범경기넥센외야수
2012KBO 시범경기두산포수
2012KBO 시범경기두산내야수
2012KBO 시범경기두산외야수
2012KBO 시범경기롯데포수
2012KBO 시범경기롯데내야수
2012KBO 시범경기롯데외야수
2012KBO 시범경기삼성포수
2012KBO 시범경기삼성내야수
2012KBO 시범경기삼성외야수
2012KBO 시범경기한화포수
2012KBO 시범경기한화내야수
2012KBO 시범경기한화외야수
2012KBO 시범경기KIA포수
2012KBO 시범경기KIA내야수
2012KBO 시범경기KIA외야수
2012KBO 시범경기LG포수
2012KBO 시범경기LG내야수
2012KBO 시범경기LG외야수
2012KBO 시범경기SK포수
2012KBO 시범경기SK내야수
2012KBO 시범경기SK외야수
2012KBO 와일드카드넥센포수
2012KBO 와일드카드넥센내야수
2012KBO 와일드카드넥센외야수
2012KBO 와일드카드두산포수
2012KBO 와일드카드두산내야수
2012KBO 와일드카드두산외야수
2012KBO 와일드카드롯데포수
2012KBO 와일드카드롯데내

KeyboardInterrupt: 